In [1]:

import json
import shutil
import os
import pickle
from callback import MultipleClassAUROC, MultiGPUModelCheckpoint
from configparser import ConfigParser
from generator import AugmentedImageSequence
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import multi_gpu_model
from utility import get_sample_counts
from weights import get_class_weights
from augmenter import augmenter
from tensorflow.keras import backend as K
import pandas as pd
import numpy as np
import time
import tensorflow.keras.initializers
import statistics
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, InputLayer, Flatten, Input, GaussianNoise
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from keras_radam import RAdam
from sklearn.metrics import confusion_matrix

%load_ext tensorboard

from datetime import datetime
from packaging import version

import tensorflow as tf
from tensorflow import keras

import numpy as np

#print("TensorFlow version: ", tf.__version__)
#assert version.parse(tf.__version__).release[0] >= 2, \
#    "This notebook requires TensorFlow 2.0 or above."


Using TensorFlow backend.


In [2]:
import gc

In [3]:
import random 
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
# UPDATED: import from tensorflow.keras instead of keras
from tensorflow.keras import layers, optimizers, losses, metrics
import gc
import tensorflow_addons as tfa

In [4]:
oneClass = "Pneumonia"

In [5]:
config_file = "./config.ini"
cp = ConfigParser()
cp.read(config_file)

    # default config
output_dir = cp["DEFAULT"].get("output_dir")
image_source_dir = cp["DEFAULT"].get("image_source_dir")
base_model_name = cp["DEFAULT"].get("base_model_name")
class_names = cp["DEFAULT"].get("class_names").split(",")

    # train config
use_base_model_weights = cp["TRAIN"].getboolean("use_base_model_weights")
use_trained_model_weights = cp["TRAIN"].getboolean("use_trained_model_weights")
use_best_weights = cp["TRAIN"].getboolean("use_best_weights")
output_weights_name = cp["TRAIN"].get("output_weights_name")
epochs = cp["TRAIN"].getint("epochs")
batch_size = cp["TRAIN"].getint("batch_size")
initial_learning_rate = cp["TRAIN"].getfloat("initial_learning_rate")
generator_workers = cp["TRAIN"].getint("generator_workers")
image_dimension = cp["TRAIN"].getint("image_dimension")
train_steps = cp["TRAIN"].get("train_steps")
patience_reduce_lr = cp["TRAIN"].getint("patience_reduce_lr")
min_lr = cp["TRAIN"].getfloat("min_lr")
validation_steps = cp["TRAIN"].get("validation_steps")
positive_weights_multiply = cp["TRAIN"].getfloat("positive_weights_multiply")
dataset_csv_dir = cp["TRAIN"].get("dataset_csv_dir")

In [6]:
def focal_loss(gamma=1.0, alpha=0.5):
    gamma = float(gamma)
    alpha = float(alpha)
    def focal_loss_fixed(y_true, y_pred):
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1.0-epsilon)
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
        return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1))-K.sum((1-alpha) * K.pow( pt_0, gamma) * K.log(1. - pt_0))
    return focal_loss_fixed

In [7]:
from tensorflow.keras.losses import Huber

In [8]:
def euclidean_distance_loss(y_true, y_pred):
    return K.sqrt(K.sum(K.square(tf.cast(y_pred,tf.float32) - tf.cast(y_true,tf.float32)), axis=-1))

In [9]:
def construct_network1(dropout=0.08425517073874295, neuronPct=0.1767547775828121, neuronShrink=0.33180474398878285):
    # We start with some percent of 5000 starting neurons on the first hidden layer.
    neuronCount = int(neuronPct * 5000)
    # Construct neural network
    neuronCount = neuronCount * neuronShrink
    model = Sequential()
    model.add(Input(shape=(1,1536)))
    model.add(Flatten(name='flat1'))
    model.add(Dense(neuronCount,name='dense1'))
    model.add(Activation('relu',name='relu1'))
    model.add(Dropout(dropout, name='dropout1'))
    model.add(Dense(14, activation='sigmoid',name='midLayer1')) # Output
    weights_path=None
    if weights_path is not None:
        print(f"load model weights_path: {weights_path}")
        model.load_weights(weights_path)
    model.layers.pop()
    dr = model.layers[-2].output
    model.trainable = False
    left = Dense(14, activation="sigmoid", name='leftLayer1')(dr)
    right = Dense(14, activation="sigmoid", name='rightLayer1')(dr)
    model = Model(model.input, [left,model.output,right])
    #model = Model(model.input, model.output)
    return model

In [10]:
def construct_network2(dropout=0.15672137551441198, neuronPct=0.2197894476507525, neuronShrink=0.3803316528497302, noisePct=0.282563134185142):
    # We start with some percent of 5000 starting neurons on the first hidden layer.
    neuronCount = int(neuronPct * 5000)
    # Construct neural network
    neuronCount = neuronCount * neuronShrink
    model = Sequential()
    model.add(Input(shape=(1,1536)))
    model.add(Flatten(name='flat2'))
    model.add(Dense(neuronCount,name='dense2'))
    model.add(GaussianNoise(noisePct))
    model.add(Activation('relu',name='relu2'))
    model.add(Dropout(dropout, name='dropout2'))
    model.add(Dense(14, activation='sigmoid',name='midLayer2')) # Output
    weights_path=None
    if weights_path is not None:
        print(f"load model weights_path: {weights_path}")
        model.load_weights(weights_path)
    #model.layers.pop()
    dr = model.layers[-2].output
    model.trainable = False
    left = Dense(14, activation="sigmoid", name='leftLayer2')(dr)
    right = Dense(14, activation="sigmoid", name='rightLayer2')(dr)
    model = Model(model.input, [left,model.output,right])
    #model = Model(model.input, model.output)
    return model

In [11]:
def construct_network(model1,model2):
    model = Model([model1.input,model2.input], [model1.output,model2.output])
    return model 

In [12]:
# compute steps
train_counts, train_pos_counts = get_sample_counts(output_dir, "train"+oneClass, class_names)
dev_counts, _ = get_sample_counts(output_dir, "dev"+oneClass, class_names)
    
if train_steps == "auto":
    train_steps = int(train_counts / batch_size)
else:
    try:
        train_steps = int(train_steps)
    except ValueError:
        raise ValueError(f"""train_steps: {train_steps} is invalid,please use 'auto' or integer.""")
    print(f"** train_steps: {train_steps} **")

if validation_steps == "auto":
    validation_steps = int(dev_counts / batch_size)
else:
    try:
        validation_steps = int(validation_steps)
    except ValueError:
        raise ValueError(f"""validation_steps: {validation_steps} is invalid,please use 'auto' or integer.""")
        print(f"** validation_steps: {validation_steps} **")

        # compute class weights
keras.backend.clear_session()
print("** compute class weights from training data **")
class_weights = get_class_weights(train_counts,train_pos_counts,multiply=positive_weights_multiply,)
print("** class_weights **")
print(class_weights)
#print(str(train_steps))
#print(str(train_counts))
#print(str(batch_size))

** compute class weights from training data **
188: 978
21: 978
195: 978
414: 978
42: 978
41: 978
978: 978
23: 978
86: 978
246: 978
16: 978
10: 978
34: 978
2: 978
** class_weights **
[{0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}]


In [13]:
test_steps = cp["TEST"].get("test_steps")
test_counts, _ = get_sample_counts(output_dir, "test", class_names)

if test_steps == "auto":
    test_steps = int(test_counts / batch_size)
else:
    try:
        test_steps = int(test_steps)
    except ValueError:
        raise ValueError(f"""test_steps: {test_steps} is invalid,please use 'auto' or integer.""")
        
print(f"** test_steps: {test_steps} **")

** test_steps: 22433 **


In [14]:
train_sequence = AugmentedImageSequence(
            dataset_csv_file=os.path.join(output_dir, "train"+oneClass+".csv"),
            class_names=class_names,
            source_image_dir=image_source_dir,
            batch_size=batch_size,
            target_size=(image_dimension, image_dimension),
            augmenter=augmenter,
            steps=train_steps,
        )
validation_sequence = AugmentedImageSequence(
            dataset_csv_file=os.path.join(output_dir, "dev"+oneClass+".csv"),
            class_names=class_names,
            source_image_dir=image_source_dir,
            batch_size=batch_size,
            target_size=(image_dimension, image_dimension),
            augmenter=augmenter,
            steps=validation_steps,
            shuffle_on_epoch_end=False,
)

test_sequence = AugmentedImageSequence(
        dataset_csv_file=os.path.join(output_dir, "test.csv"),
        class_names=class_names,
        source_image_dir=image_source_dir,
        batch_size=batch_size,
        target_size=(image_dimension, image_dimension),
        augmenter=None,
        steps=test_steps,
        shuffle_on_epoch_end=False,
    )

In [15]:
def optimize_network(lr):
    gc.collect()
      # Define the Keras TensorBoard callback.
    logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")

    start_time = time.time()
    model1 = construct_network1()
    model2 = construct_network2()
    
    optimizer = SGD(lr=initial_learning_rate)
    
    alpha = 0.9340456763831478
    gamma = 1.4195808780694898
    model1.compile(optimizer=optimizer,loss={'leftLayer1':tf.keras.losses.Huber(),'midLayer1':focal_loss(gamma=gamma,alpha=alpha),'rightLayer1':euclidean_distance_loss})

    alpha = 0.7297456293468533
    gamma = 1.2700405014991505
    model2.compile(optimizer=optimizer,loss={'leftLayer2':tf.keras.losses.Huber(),'midLayer2':focal_loss(gamma=gamma,alpha=alpha),'rightLayer2':euclidean_distance_loss})
  
    model = construct_network(model1=model1,model2=model2)
    model.compile(optimizer=optimizer,loss={'leftLayer1':tf.keras.losses.Huber(),'midLayer1':focal_loss(gamma=gamma,alpha=alpha),'rightLayer1':euclidean_distance_loss,'leftLayer2':tf.keras.losses.Huber(),'midLayer2':focal_loss(gamma=gamma,alpha=alpha),'rightLayer2':euclidean_distance_loss})

    output_weights_path = os.path.join(output_dir,  str(lr)+"_"+output_weights_name)
    
    print(f"** set output weights path to: {output_weights_path} **")
                  
    
                  
    checkpoint = ModelCheckpoint(
                 output_weights_path,
                 save_weights_only=True,
                 save_best_only=True,
                 verbose=1,
            )
    start_time = time.time()
  
    model.summary()
  
    callbacks = [
            checkpoint,
            #keras.callbacks.TensorBoard(log_dir=logdir),
            #TensorBoard(log_dir=os.path.join(output_dir, "logs"), batch_size=batch_size),
            ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=patience_reduce_lr,
                              verbose=1, mode="min", min_lr=min_lr), 
            EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto', restore_best_weights=True)
    ]
    
    
    history = model.fit_generator(
            generator=train_sequence,
            steps_per_epoch=train_steps,
            epochs=epochs,
            validation_data=validation_sequence,
            validation_steps=validation_steps,
            callbacks=callbacks,
            class_weight=[class_weights,class_weights,class_weights,class_weights,class_weights,class_weights],
            workers=generator_workers,
            shuffle=False,
        )
        
    y_hat = model.predict_generator(test_sequence, verbose=1)
    y = test_sequence.get_y_true()
    
    test_log_path = os.path.join(output_dir, str(lr)+"_"+"test.log")
    print(f"** write log to {test_log_path} **")
    aurocs = []
    auprcs = []
    precision = dict()
    recall = dict()
    threshold = dict()
    with open(test_log_path, "w") as f:
        for k in range(6):
            for i in range(len(class_names)):
                 if(class_names[i] == str(oneClass)):
                
                    try:
                        score = roc_auc_score(y[:, i], y_hat[k][:, i])
                        precision[i], recall[i], threshold[i] = precision_recall_curve(y[:, i], y_hat[k][:, i])
                        tmp = auc(recall[i], precision[i])
                        aurocs.append(score)
                        auprcs.append(tmp) 
                    except ValueError:
                        score = 0
               
                    print(f"auroc {str(k)+class_names[i]}: {score}\n")
                    print(f"auprc {str(k)+class_names[i]}: {tmp}\n")
                    f.write(f"auroc {str(k)+class_names[i]}: {score}\n")
                    f.write(f"auprc {str(k)+class_names[i]}: {tmp}\n")
        
        mean_auroc = np.mean(aurocs)
        mean_auprc = float(np.mean(auprcs))
        f.write("-------------------------\n")
        f.write(f"mean auroc: {mean_auroc}\n")
        print(f"mean auroc: {mean_auroc}\n")
        f.write(f"mean auprc: {mean_auprc}\n")
        print(f"mean auprc: {mean_auprc}\n")
        
        max_auroc = np.max(aurocs)
        max_auprc = float(np.max(auprcs))
        f.write("-------------------------\n")
        f.write(f"max auroc: {max_auroc}\n")
        print(f"max auroc: {max_auroc}\n")
        f.write(f"max auprc: {max_auprc}\n")
        print(f"max auprc: {max_auprc}\n")
    
    keras.backend.clear_session()
    time_took = time.time() - start_time
    
    return max_auroc, time_took
    


In [16]:
step = np.arange(0.009, 0.0151, 0.001)
maxi = []
for k in np.nditer(step):
    opn, daTime = optimize_network(k)
    print(daTime)
    maxi.append(opn)




** set output weights path to: ./experiments/0.009_weights.h5 **
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 1, 1536)]    0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 1, 1536)]    0                                            
__________________________________________________________________________________________________
flat2 (Flatten)                 (None, 1536)         0           input_2[0][0]                    
__________________________________________________________________________________________________
flat1 (Flatten)                 (None, 1536)         0           input_1[0][0]                    
___________________________

Epoch 6/11
970/978 [============================>.] - ETA: 0s - loss: 5.5960 - leftLayer1_loss: 0.1155 - midLayer1_loss: 1.4418 - rightLayer1_loss: 1.3711 - leftLayer2_loss: 0.0996 - midLayer2_loss: 1.4882 - rightLayer2_loss: 1.0798
Epoch 00006: val_loss improved from 5.74561 to 5.64270, saving model to ./experiments/0.009_weights.h5
978/978 [==============================] - 2s 3ms/step - loss: 5.5941 - leftLayer1_loss: 0.1155 - midLayer1_loss: 1.4416 - rightLayer1_loss: 1.3705 - leftLayer2_loss: 0.0996 - midLayer2_loss: 1.4878 - rightLayer2_loss: 1.0791 - val_loss: 5.6427 - val_leftLayer1_loss: 0.1153 - val_midLayer1_loss: 1.4336 - val_rightLayer1_loss: 1.3365 - val_leftLayer2_loss: 0.1069 - val_midLayer2_loss: 1.3942 - val_rightLayer2_loss: 1.2562
Epoch 7/11
959/978 [============================>.] - ETA: 0s - loss: 5.5009 - leftLayer1_loss: 0.1137 - midLayer1_loss: 1.4425 - rightLayer1_loss: 1.3184 - leftLayer2_loss: 0.0952 - midLayer2_loss: 1.4839 - rightLayer2_loss: 1.0472
Epoch 

  {'leftLayer1': '...', 'midLayer1': '...', 'rightLayer1': '...', 'leftLayer2': '...', 'midLayer2': '...', 'rightLayer2': '...'}
    to  
  ['...', '...', '...', '...', '...', '...']
Train for 978 steps, validate for 133 steps
Epoch 1/11
970/978 [============================>.] - ETA: 0s - loss: 6.7423 - leftLayer1_loss: 0.1290 - midLayer1_loss: 1.3798 - rightLayer1_loss: 1.8219 - leftLayer2_loss: 0.1170 - midLayer2_loss: 1.5599 - rightLayer2_loss: 1.7347
Epoch 00001: val_loss improved from inf to 6.51582, saving model to ./experiments/0.009999999999999998_weights.h5
978/978 [==============================] - 3s 3ms/step - loss: 6.7402 - leftLayer1_loss: 0.1289 - midLayer1_loss: 1.3797 - rightLayer1_loss: 1.8214 - leftLayer2_loss: 0.1169 - midLayer2_loss: 1.5601 - rightLayer2_loss: 1.7331 - val_loss: 6.5158 - val_leftLayer1_loss: 0.1284 - val_midLayer1_loss: 1.3749 - val_rightLayer1_loss: 1.7514 - val_leftLayer2_loss: 0.1179 - val_midLayer2_loss: 1.4801 - val_rightLayer2_loss: 1.6631
E

966/978 [============================>.] - ETA: 0s - loss: 5.2555 - leftLayer1_loss: 0.1065 - midLayer1_loss: 1.3805 - rightLayer1_loss: 1.1337 - leftLayer2_loss: 0.0761 - midLayer2_loss: 1.5618 - rightLayer2_loss: 0.9970
Epoch 00011: val_loss improved from 5.31727 to 5.27622, saving model to ./experiments/0.009999999999999998_weights.h5
22433/22433 [==============================] - 33s 1ms/step
** write log to ./experiments/0.009999999999999998_test.log **
auroc 0Pneumonia: 0.5547280913154056

auprc 0Pneumonia: 0.01602681990259543

auroc 1Pneumonia: 0.5454727942345773

auprc 1Pneumonia: 0.014795445276586204

auroc 2Pneumonia: 0.5206988649631245

auprc 2Pneumonia: 0.011223698187168457

auroc 3Pneumonia: 0.43368281609214665

auprc 3Pneumonia: 0.00898398870752167

auroc 4Pneumonia: 0.6096263804364139

auprc 4Pneumonia: 0.01632706562177243

auroc 5Pneumonia: 0.42720487160493553

auprc 5Pneumonia: 0.008953250072646845

mean auroc: 0.5152356364411006

mean auprc: 0.01271837796138184

max a

Epoch 4/11
960/978 [============================>.] - ETA: 0s - loss: 5.7191 - leftLayer1_loss: 0.1189 - midLayer1_loss: 1.3310 - rightLayer1_loss: 1.4904 - leftLayer2_loss: 0.1040 - midLayer2_loss: 1.5129 - rightLayer2_loss: 1.1619
Epoch 00004: val_loss improved from 5.87780 to 5.71681, saving model to ./experiments/0.010999999999999998_weights.h5
978/978 [==============================] - 3s 3ms/step - loss: 5.7154 - leftLayer1_loss: 0.1189 - midLayer1_loss: 1.3310 - rightLayer1_loss: 1.4895 - leftLayer2_loss: 0.1039 - midLayer2_loss: 1.5114 - rightLayer2_loss: 1.1607 - val_loss: 5.7168 - val_leftLayer1_loss: 0.1175 - val_midLayer1_loss: 1.3230 - val_rightLayer1_loss: 1.4501 - val_leftLayer2_loss: 0.1104 - val_midLayer2_loss: 1.4145 - val_rightLayer2_loss: 1.3014
Epoch 5/11
975/978 [============================>.] - ETA: 0s - loss: 5.5640 - leftLayer1_loss: 0.1171 - midLayer1_loss: 1.3300 - rightLayer1_loss: 1.4125 - leftLayer2_loss: 0.0995 - midLayer2_loss: 1.5016 - rightLayer2_loss

...

962/978 [============================>.] - ETA: 0s - loss: 5.1226 - leftLayer1_loss: 0.0994 - midLayer1_loss: 1.2981 - rightLayer1_loss: 1.1131 - leftLayer2_loss: 0.0837 - midLayer2_loss: 1.5367 - rightLayer2_loss: 0.9915
Epoch 00011: val_loss improved from 5.16101 to 5.12141, saving model to ./experiments/0.011999999999999997_weights.h5
22433/22433 [==============================] - 32s 1ms/step
** write log to ./experiments/0.011999999999999997_test.log **
auroc 0Pneumonia: 0.4819689949503019

auprc 0Pneumonia: 0.010831199054053307

auroc 1Pneumonia: 0.48065927255893703

auprc 1Pneumonia: 0.009722917523186244

auroc 2Pneumonia: 0.43382750284811317

auprc 2Pneumonia: 0.009109151051385796

auroc 3Pneumonia: 0.5119435472127596

auprc 3Pneumonia: 0.011025147612386894

auroc 4Pneumonia: 0.5579096171443191

auprc 4Pneumonia: 0.012776627128304413

auroc 5Pneumonia: 0.5368356280243163

auprc 5Pneumonia: 0.011221600061604095

mean auroc: 0.5005240937897911

mean auprc: 0.010781107071820126

m

Epoch 4/11
965/978 [============================>.] - ETA: 0s - loss: 5.8504 - leftLayer1_loss: 0.1131 - midLayer1_loss: 1.4558 - rightLayer1_loss: 1.4160 - leftLayer2_loss: 0.1101 - midLayer2_loss: 1.5775 - rightLayer2_loss: 1.1779
Epoch 00004: val_loss improved from 5.97764 to 5.80599, saving model to ./experiments/0.012999999999999996_weights.h5
978/978 [==============================] - 4s 4ms/step - loss: 5.8499 - leftLayer1_loss: 0.1131 - midLayer1_loss: 1.4558 - rightLayer1_loss: 1.4152 - leftLayer2_loss: 0.1101 - midLayer2_loss: 1.5781 - rightLayer2_loss: 1.1777 - val_loss: 5.8060 - val_leftLayer1_loss: 0.1115 - val_midLayer1_loss: 1.4346 - val_rightLayer1_loss: 1.3598 - val_leftLayer2_loss: 0.1146 - val_midLayer2_loss: 1.4423 - val_rightLayer2_loss: 1.3431
Epoch 5/11
963/978 [============================>.] - ETA: 0s - loss: 5.7028 - leftLayer1_loss: 0.1108 - midLayer1_loss: 1.4571 - rightLayer1_loss: 1.3356 - leftLayer2_loss: 0.1047 - midLayer2_loss: 1.5756 - rightLayer2_loss

  {'leftLayer1': '...', 'midLayer1': '...', 'rightLayer1': '...', 'leftLayer2': '...', 'midLayer2': '...', 'rightLayer2': '...'}
    to  
  ['...', '...', '...', '...', '...', '...']
Train for 978 steps, validate for 133 steps
Epoch 1/11
975/978 [============================>.] - ETA: 0s - loss: 6.7955 - leftLayer1_loss: 0.1333 - midLayer1_loss: 1.4460 - rightLayer1_loss: 1.8062 - leftLayer2_loss: 0.1206 - midLayer2_loss: 1.5905 - rightLayer2_loss: 1.6988
Epoch 00001: val_loss improved from inf to 6.56891, saving model to ./experiments/0.013999999999999995_weights.h5
978/978 [==============================] - 4s 4ms/step - loss: 6.7951 - leftLayer1_loss: 0.1333 - midLayer1_loss: 1.4462 - rightLayer1_loss: 1.8059 - leftLayer2_loss: 0.1206 - midLayer2_loss: 1.5907 - rightLayer2_loss: 1.6983 - val_loss: 6.5689 - val_leftLayer1_loss: 0.1324 - val_midLayer1_loss: 1.4518 - val_rightLayer1_loss: 1.7360 - val_leftLayer2_loss: 0.1201 - val_midLayer2_loss: 1.4662 - val_rightLayer2_loss: 1.6623
E

973/978 [============================>.] - ETA: 0s - loss: 5.3676 - leftLayer1_loss: 0.1106 - midLayer1_loss: 1.4470 - rightLayer1_loss: 1.1359 - leftLayer2_loss: 0.0806 - midLayer2_loss: 1.5987 - rightLayer2_loss: 0.9946
Epoch 00011: val_loss improved from 5.43173 to 5.38986, saving model to ./experiments/0.013999999999999995_weights.h5
22433/22433 [==============================] - 29s 1ms/step
** write log to ./experiments/0.013999999999999995_test.log **
auroc 0Pneumonia: 0.4013469089359807

auprc 0Pneumonia: 0.008203007100559262

auroc 1Pneumonia: 0.4397747430180726

auprc 1Pneumonia: 0.008794083145345237

auroc 2Pneumonia: 0.5185016559836819

auprc 2Pneumonia: 0.011050883014993226

auroc 3Pneumonia: 0.47723474001633454

auprc 3Pneumonia: 0.009608791213055976

auroc 4Pneumonia: 0.5534654805704494

auprc 4Pneumonia: 0.012052258824981375

auroc 5Pneumonia: 0.435903487788773

auprc 5Pneumonia: 0.009522799075948385

mean auroc: 0.4710378360522154

mean auprc: 0.00987197039581391

max 

Epoch 4/11
960/978 [============================>.] - ETA: 0s - loss: 5.8595 - leftLayer1_loss: 0.1163 - midLayer1_loss: 1.4288 - rightLayer1_loss: 1.4648 - leftLayer2_loss: 0.1121 - midLayer2_loss: 1.5647 - rightLayer2_loss: 1.1728
Epoch 00004: val_loss improved from 6.01561 to 5.84054, saving model to ./experiments/0.014999999999999994_weights.h5
978/978 [==============================] - 3s 3ms/step - loss: 5.8607 - leftLayer1_loss: 0.1163 - midLayer1_loss: 1.4301 - rightLayer1_loss: 1.4638 - leftLayer2_loss: 0.1121 - midLayer2_loss: 1.5662 - rightLayer2_loss: 1.1722 - val_loss: 5.8405 - val_leftLayer1_loss: 0.1152 - val_midLayer1_loss: 1.4224 - val_rightLayer1_loss: 1.4066 - val_leftLayer2_loss: 0.1130 - val_midLayer2_loss: 1.4848 - val_rightLayer2_loss: 1.2985
Epoch 5/11
967/978 [============================>.] - ETA: 0s - loss: 5.7163 - leftLayer1_loss: 0.1140 - midLayer1_loss: 1.4309 - rightLayer1_loss: 1.3816 - leftLayer2_loss: 0.1067 - midLayer2_loss: 1.5735 - rightLayer2_loss

In [17]:
print(np.max(maxi))

0.6195349093575647
